## The .prm file—post glacial rebound model

For this model, you will need the parameter file, `'postglacial_rebound.prm'`, which is available in the current directory. This input file simulates the how an elastic lithosphere deforms under the applied ice sheet loads and the rebound of the viscous asthenosphere as the load is removed.

<div class="alert alert-block alert-info">
<b>NOTE:</b> The parameter names and their values are case-sensitive. This means that using `Function` instead of `function` will throw an error in ASPECT.
</div>

ASPECT uses SI units: meter, kg, seconds/years.

### Global Parameters
In both input .prm files, we use a lower CFL number (`0.001`) than the default value of 1. The lower CFL number ensures that the variations in the modeled velocities are not too big, leading to a stable solution.
```
# Global parameters
set Start time                             = 0
set End time                               = 100000
set Use years in output instead of seconds = true
set CFL number                             = 0.001
```

### Model Geometry
We simulate the problem in a box with dimensions $6000 \times 400$ km. 
```
subsection Geometry model
  set Model name = box

  subsection Box
    set X repetitions = 2
    set Y repetitions = 1
    set X extent      = 6000000
    set Y extent      = 400000
  end
end
```

### Simulating elastic lithosphere
The elastic rheology in ASPECT requires compositional fields corresponding to the unique stress terms in the viscoelastic stress tensor, e.g., `ve_stress_xx` represents the normal stress component in the x direction. Besides the stress components, we also have the `lithosphere` compositional field which deforms elastically under the applied stress.
``` 
subsection Compositional fields
  set Number of fields = 4
  set Names of fields  = ve_stress_xx, ve_stress_yy, ve_stress_xy, lithosphere
end
```

We define the lithospheric composition using a hyperbolic tangent function such the transition between the lithosphere and the lithospheric mantle is smooth, and the resultant lithosphere is 80 km thick with 10 km of transitional thickness.
```
subsection Initial composition model
  set Model name = function
  subsection Function
    set Variable names      = x,y
    set Function constants  = 
    set Function expression = 0; 0; 0; 0.5+0.5*tanh((y-320000)/10000)
  end
end
```

### Applied traction to represent the line load
We allow the top model surface to deform freely in response to the applied load. these loads are defined using the `Boundary traction model`, which specifies a Gaussian-shaped traction centered on the model domain for the the first 90k years of the model run. The peak amplitude of the traction corresponds to the stress exerted by an ice sheet with a maximum thickness of 3 km. A negative sign indicates that the traction acts downward—compressing the top boundary—in contrast to an outward or upward-directed force.

```
subsection Mesh deformation
  set Mesh deformation boundary indicators        = top: free surface
  set Additional tangential mesh velocity boundary indicators = left, right
  subsection Free surface
    set Surface velocity projection = vertical
    set Free surface stabilization theta = 0.75
  end
end


subsection Boundary traction model
  set Prescribed traction boundary indicators = top:function
  subsection Function
    set Variable names      = x,y,t
    set Function constants  = d=3000,l=1e6,rho=1000,g=10, x0=3e6
    set Function expression = 0; if(t<90000,-rho*g*d*exp(-(x-x0)^2/(2*l*l)),0)
  end
end
```


### Discretization
We use discontinous elements in compositional fields to avoid any oscillations as the lithosphere deforms over time. We also limit the compositional values such that they are within the expected physical bounds. 

```
subsection Discretization
  set Composition polynomial degree           = 2
  set Temperature polynomial degree           = 1
  set Use discontinuous temperature discretization = false
  set Use discontinuous composition discretization = true
  subsection Stabilization parameters
      set Use limiter for discontinuous composition solution = true
      set Global composition maximum =  1.e11,  1.e11,  1.e11, 1.0
      set Global composition minimum = -1.e11, -1.e11, -1.e11, 0.0
  end
end
```

###  Material model 

To include elasticity, we need to use elastic terms in the right hand side of the momentum equation using the `Formulation` subsection.
```
subsection Formulation
  set Enable elasticity = true
end
```

Our model is viscoelastic such that the lithosphere deforms elastically, i.e., has a lower elastic shear modulus of 70e9, while the lithospheric mantle responds by flowing viscously around it, governed by the low viscosity of 5e19 Pa-s. 
```
subsection Material model

  set Model name = viscoelastic

  subsection Viscoelastic
    set Densities            = 3300
    set Viscosities          = 5.e19, 1.e24, 1.e24, 1.e24, 1.e24
    set Elastic shear moduli = 1.e12, 1.e12, 1.e12, 1.e12, 70.e9
    set Fixed elastic time step     = 1e3
    set Use fixed elastic time step = false
    set Viscosity averaging scheme  = maximum composition
    set Thermal expansivities = 0
  end

end
```


## The .prm file—ocean island loading model

For this model, you will need the parameter file, `'ocean_island_loading.prm'`, which is available in the current directory. This input file simulates the how an elastic lithosphere deforms under the weight of the oceanic islands.

The .prm files have similar structure as the `postglacial_rebound.prm` problem, therefore for this model, we will only look at the sections specific to defining this problem.

### Defining the oceanic lithosphere

Similar to above, we use a hyperbolic tangent function to define the lithosphere with a transition thickness of 10 km to the lithospheric mantle. We approximate the oceanic lithosphere of 34 km thickness using this function.

```
subsection Initial composition model
  set Model name = function
  subsection Function
    set Variable names      = x,y
    set Function constants  = y0=400000, lithosphere_thickness=34000
    set Function expression = 0; 0; 0; 0.5+0.5*tanh((y-y0+lithosphere_thickness)/10000)
  end
end
```

### Approximating the applied oceanic island load
We approximate the oceanic island load using aGgausssian function such that the maximum applitude corresponds to an 8 km high basalt submerged in water. Contrary to the post-glacial rebound model, the applied load is constant over time.

```
subsection Boundary traction model
  set Prescribed traction boundary indicators = top:function
  subsection Function
    set Variable names      = x,y,t
    set Function constants  = d=8000,l=2.5e4,rho=2300,g=10, x0=4e5
    set Function expression = 0; -rho*g*d*exp(-(x-x0)^2/(2*l*l))
  end
end
```

 &nbsp;<div style="text-align: right">  
    &rarr; <b>NEXT: [Plotting the model simulation results](./4_plotting_the_model_simulation_results.ipynb) </b> <a href=""></a> &nbsp;&nbsp;
     <img src="../assets/education-gem-notebooks_icon.png" alt="icon"  style="width:4%">
  </div>